In [1]:
print('Starts.')

Starts.


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
import os
os.environ['http_proxy'] = 'http://192.168.1.108:7890'
os.environ['https_proxy'] = 'http://192.168.1.108:7890'

! curl -I www.google.com

In [4]:

# 加载预训练模型和分词器
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#model = AutoModel.from_pretrained("bert-base-uncased")

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModel.from_pretrained("roberta-base")


def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state[:, 0, :]  # 使用[CLS] token的嵌入
    return embeddings.squeeze().cpu().numpy()


/home/xiaomai/anaconda3/envs/aikb/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
KB_file = "./knowledge_base.txt"

# 读取文本数据并生成嵌入
with open(KB_file, "r") as file:
    texts = file.readlines()
embeddings = [embed_text(text) for text in texts]

In [13]:
#  使用FAISS存储和检索向量：
import faiss
import numpy as np

# 创建FAISS索引
dimension = 768  # 嵌入维度
index = faiss.IndexFlatL2(dimension)
embeddings_np = np.array(embeddings, dtype=np.float32)
index.add(embeddings_np)

# 保存索引
faiss.write_index(index, "knowledge_base.index")


In [14]:
#检查FAISS索引是否正确保存和加载：
index = faiss.read_index("knowledge_base.index")
query = embed_text("How many data modelling examples?")
D, I = index.search(np.array([query], dtype=np.float32), k=5)  # 搜索前5个最相似的
print(I)  # 输出匹配的索引


[[51 47 43 55  0]]


In [17]:
#编写查询函数，返回最匹配的文本：
def search(query_text, top_k=5):
    query_embedding = embed_text(query_text)
    D, I = index.search(np.array([query_embedding], dtype=np.float32), k=top_k)
    results = [texts[i] for i in I[0]]
    return results

# 测试查询
results = search("What is the data model process? Please list the steps.", top_k=5)
#print(results)
results


['By modeling your data, you can document what types of data you have, how you use it, and the data management requirements surrounding its usage, protection, and governance. The benefits of data modeling include:\n',
 "it's not just about the results of data modeling, but how you get those results.\n",
 'Now that you know what data modeling is and why it’s important, let’s look at the three different types of data modeling concepts as examples.\n',
 'Saving time and money on IT and process investments through appropriate planning.\n',
 'Creating a structure for collaboration between your IT and business teams.\n']